**Author**: Yap Jheng Khin

**FYP II Title**: Used car dealership web application

**Purpose**:
1. This notebook automates the creation of classes in C# code. This is neccessary since each new record needs the prediction from the model. This program also inserts application test data that are neccessary to conduct the system testing.
2. For car price analytics, each car inventory record requires a non-empty field named `PredictedPrice`. In the web application, the `PredictedPrice` field value will be updated each time the users add or update any car inventory record. 
3. For lead scoring, each lead information requires a non-empty field named `Status`. In the web application, the `Status`'s initial value is `Active` when the users add the record. The users will set the `Status` to `Qualified` if the lead conversion is successful and set to `Disqualified` if the opposite case is true.

- Input: 
    - Cleaned lead scoring dataset.
    - Cleaned car price dataset.
    - Lead scoring model used in the web service.
    - Car price model used in the web service.
- Ouput
    - Files containing the C# code to initialize Lead class, CarBrand class, CarModel class, and Car class.

**Execution time**: At most 1 minute in Jupyter Notebook.

# Setup

Ensure that the current Python interpreter path is correct. For example, if the **River conda environment** is named as **arf_conda_env**, the expected `sys.executable` should be C:\Users\User\miniconda3\envs\\**arf_conda_env**\\python.exe.

In [1]:
import sys
print(sys.executable)

C:\Users\User\miniconda3\envs\arf_conda_env\python.exe


In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import math
import re

In [3]:
RANDOM_SEED = 2022
INPUT_FOLDER_PATH = 'inputs'

OUT_PARENT_DIR = 'outputs'
OUT_CHILD_DIR = 'seed_data'
OUT_FOLDER_PATH = os.path.join(OUT_PARENT_DIR, OUT_CHILD_DIR) 

# Create directory if not exist
os.makedirs(OUT_FOLDER_PATH, exist_ok=True)

# Lead Scoring

## Load data

The data was transfered from the excel file to the web application in the SeedData.cs. Code was written to automate the transfer.

In [4]:
ls_train = pd.read_csv(os.path.join(OUT_PARENT_DIR, 'lead_scoring', 'train_set.csv'))
ls_test  = pd.read_csv(os.path.join(OUT_PARENT_DIR, 'lead_scoring', 'test_set.csv'))

ls_test.head(5)

,dont_email,dont_call,occupation,received_free_copy,total_site_visit,total_time_spend_on_site,avg_page_view_per_visit,converted
0,No,No,Unemployed,No,0.0,0.0,0.0,1
1,No,No,Working Professional,No,0.0,0.0,0.0,1
2,No,No,Unemployed,Yes,5.0,253.0,5.0,1
3,No,No,Student,No,0.0,0.0,0.0,0
4,No,No,Unemployed,No,9.0,87.0,4.5,1


In [5]:
ls_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1305 entries, 0 to 1304
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   dont_email                1305 non-null   object 
 1   dont_call                 1305 non-null   object 
 2   occupation                1305 non-null   object 
 3   received_free_copy        1305 non-null   object 
 4   total_site_visit          1305 non-null   float64
 5   total_time_spend_on_site  1305 non-null   float64
 6   avg_page_view_per_visit   1305 non-null   float64
 7   converted                 1305 non-null   int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 81.7+ KB


## Initialize Lead

Below is the lookup table to transfer from Python syntax to C# syntax.

In [6]:
ls_table = {
    "ID":{"type":"int"},
    'DontEmail': {
        'No': 'DontEmailEnum.No', 
        'Yes': 'DontEmailEnum.Yes'
    },
    'DontCall': {
        'No': 'DontCallEnum.No', 
        'Yes': 'DontCallEnum.Yes'
    },
    'Occupation': {
        'Unemployed': 'OccupationEnum.Unemployed', 
        'Student': 'OccupationEnum.Student', 
        'Businessman': 'OccupationEnum.Businessman', 
        'Working Professional': 'OccupationEnum.WorkingProfessional'
    },
    'ReceivedFreeCopy': {
        'No': 'ReceivedFreeCopyEnum.No', 
        'Yes': 'ReceivedFreeCopyEnum.Yes'
    },
    'Status': {
        '1': 'StatusEnum.Qualified',
        '0': 'StatusEnum.Disqualified',
        'Active': 'StatusEnum.Active'
    }
}

`Lead` class syntax was translated from the Python dataframe using the command below. Then, the output was copied to the application to initialise new instances of `Lead`. The output was saved to `init_leads.txt`.

In [7]:
# Load pretrained adaptive random forest
with open(os.path.join(OUT_PARENT_DIR, 'lead_scoring', 'arf_cf.pkl'), 'rb') as f:
    lead_scoring_model = pickle.load(f)

# Load adaptive random forest that is trained from scratch    
with open(os.path.join(OUT_PARENT_DIR, 'lead_scoring', 'data_preprocessor.pkl'), 'rb') as f:
    cp_data_pp = pickle.load(f)

# Get the predicted ls price
ls_X_test = ls_test.drop(columns='converted', axis=1)
ls_X_test_pp = cp_data_pp.preprocess(ls_X_test)
pred_scores = []
for index, row in ls_X_test_pp.iterrows():
    pred_scores.append(lead_scoring_model.predict_proba_one(row)[1.0])
ls_test['pred_score'] = np.array(pred_scores)

# Randomly assigned values to the 'update_analytics' column
# In the web application, if the users want to update the ls price analytics 
# with the current ls inventory record, the users must select the option 
# 'Update the current price to price analytics'. 
# The option is randomly chosen during the initialization process to test the system.
lead_status = np.empty(len(ls_test), dtype=object)

# Define a random number generator object
# Source: https://towardsdatascience.com/stop-using-numpy-random-seed-581a9972805f
rng = np.random.default_rng(RANDOM_SEED)

for target_class, target_count in ls_test['converted'].value_counts().iteritems():
    # Qualified or disqualified counts
    non_active_counts = math.floor(target_count/2)
    active_counts = math.ceil(target_count/2)
    cur_lead_status = np.append(
        np.full(non_active_counts, str(target_class)),
        np.full(active_counts, 'Active')
    )
    rng.shuffle(cur_lead_status)
    lead_status[ls_test['converted'] == target_class] = cur_lead_status
    
ls_test['lead_status'] = lead_status

# Export the dataframe to be used in SHAP tree explainer
ls_test.to_csv(os.path.join(OUT_FOLDER_PATH, 'lead_records.csv'), index=False)

ls_test.head(5)

,dont_email,dont_call,occupation,received_free_copy,total_site_visit,total_time_spend_on_site,avg_page_view_per_visit,converted,pred_score,lead_status
0,No,No,Unemployed,No,0.0,0.0,0.0,1,0.570864,Active
1,No,No,Working Professional,No,0.0,0.0,0.0,1,0.930392,Active
2,No,No,Unemployed,Yes,5.0,253.0,5.0,1,0.203909,Active
3,No,No,Student,No,0.0,0.0,0.0,0,0.319241,0
4,No,No,Unemployed,No,9.0,87.0,4.5,1,0.245517,Active


In [8]:
ls_test['lead_status'].value_counts()

Active    653
0         352
1         300
Name: lead_status, dtype: int64

Retrieve the names, email addresses and phone numbers from the files. The information is totally fake and generated from random generator website.

In [9]:
with open("inputs/lead_names_random_generated.txt", 'r') as f:
    lead_names = f.readlines()

with open("inputs/lead_emails_random_generated.txt", 'r') as f:
    emails = f.readlines()

with open("inputs/lead_phone_no_random_generated.txt", 'r') as f:
    phone_numbers = f.readlines()

In [10]:
with open(os.path.join(OUT_FOLDER_PATH, 'init_leads.txt'), 'w') as f:
    for iter_no, (index, row) in enumerate(ls_test.iterrows()):
        lead_init_code = f"""
        new Lead
        {{
            Name = "{lead_names[iter_no].strip()}",
            Email = "{emails[iter_no].strip()}",
            PhoneNo = "{phone_numbers[iter_no].strip()}",
            AvgPageViewPerVisit = {row['avg_page_view_per_visit']},
            CreatedTimestamp = new DateTime(2022, 4, 1, 0, 0, 0),
            DontCall = Lead.DontCallList[(int)Lead.{ls_table['DontCall'][row['dont_call']]}],
            DontEmail = Lead.DontEmailList[(int)Lead.{ls_table['DontEmail'][row['dont_email']]}],
            Occupation = Lead.OccupationList[(int)Lead.{ls_table['Occupation'][row['occupation']]}],
            PredictedScore = {round(row['pred_score'], 4)},
            ReceivedFreeCopy = Lead.ReceivedFreeCopyList[(int)Lead.{ls_table['ReceivedFreeCopy'][row['received_free_copy']]}],
            Status = Lead.StatusList[(int)Lead.{ls_table['Status'][row['lead_status']]}],
            TotalSiteVisit = {row['total_site_visit']},
            TotalTimeSpendOnSite = {row['total_time_spend_on_site']}
        }},
        """
        f.write(f'{lead_init_code}\n')
        
print(f'Example of C# code initializing Car:\n\n\t{lead_init_code}')

Example of C# code initializing Car:

	
        new Lead
        {
            Name = "Ryan Clarke",
            Email = "ryan_clarke@gmail.com",
            PhoneNo = "019-6531076",
            AvgPageViewPerVisit = 3.0,
            CreatedTimestamp = new DateTime(2022, 4, 1, 0, 0, 0),
            DontCall = Lead.DontCallList[(int)Lead.DontCallEnum.No],
            DontEmail = Lead.DontEmailList[(int)Lead.DontEmailEnum.No],
            Occupation = Lead.OccupationList[(int)Lead.OccupationEnum.Unemployed],
            PredictedScore = 0.675,
            ReceivedFreeCopy = Lead.ReceivedFreeCopyList[(int)Lead.ReceivedFreeCopyEnum.No],
            Status = Lead.StatusList[(int)Lead.StatusEnum.Disqualified],
            TotalSiteVisit = 3.0,
            TotalTimeSpendOnSite = 1195.0
        },
        


# Car Price Prediction

The data was transfered from the excel file to the web application in the SeedData.cs. Code was written to automate the transfer.

## Load Data

In [11]:
car_train = pd.read_csv(os.path.join(OUT_PARENT_DIR, 'car_price', 'car_train_processed.csv'))
car_test  = pd.read_csv(os.path.join(OUT_PARENT_DIR, 'car_price', 'car_test_processed.csv'))

car_test.head(5)

,manufacture_year,mileage,price,length_mm,engine_cc,aspiration,wheel_base_mm,width_mm,direct_injection,seat_capacity,...,fuel_type,steering_type,assembled,height_mm,peak_torque_nm,doors,brand,colour,model,transmission
0,2006.0,95000.0,12800.0,3480.0,989.0,Aspirated,2360.0,1490.0,Multi-Point Injected,5.0,...,Petrol - Unleaded (ULP),Rack and Pinion,Locally Built,1425.0,88.0,5.0,Perodua,Silver,Kelisa,Automatic
1,2014.0,75000.0,24800.0,4618.0,1997.0,Aspirated,2702.0,1813.0,Multi-Point Injected,5.0,...,Petrol - Unleaded (ULP),Rack and Pinion,Locally Built,1485.0,195.0,4.0,Renault,Silver,Fluence,Automatic
2,2014.0,98000.0,86800.0,4705.0,2694.0,Aspirated,2750.0,1840.0,Multi-Point Injected,7.0,...,Petrol - Unleaded (ULP),Rack and Pinion,Locally Built,1850.0,241.0,5.0,Toyota,Silver,Fortuner,Automatic
3,2003.0,185000.0,8800.0,4465.0,1584.0,Aspirated,2600.0,1740.0,Multi-Point Injected,5.0,...,Petrol - Unleaded (ULP),Rack and Pinion,Locally Built,1420.0,140.0,4.0,Proton,Silver,Waja,Manual
4,2014.0,105000.0,30800.0,4536.0,1561.0,Turbocharged,2650.0,1786.0,Multi-Point Injected,5.0,...,Petrol - Unleaded (ULP),Rack and Pinion,Locally Built,1524.0,205.0,5.0,Proton,Red,Suprima S,Automatic


In [12]:
car_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094 entries, 0 to 1093
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   manufacture_year  1094 non-null   float64
 1   mileage           1094 non-null   float64
 2   price             1094 non-null   float64
 3   length_mm         1094 non-null   float64
 4   engine_cc         1094 non-null   float64
 5   aspiration        1094 non-null   object 
 6   wheel_base_mm     1094 non-null   float64
 7   width_mm          1094 non-null   float64
 8   direct_injection  1094 non-null   object 
 9   seat_capacity     1094 non-null   float64
 10  peak_power_hp     1094 non-null   float64
 11  fuel_type         1094 non-null   object 
 12  steering_type     1094 non-null   object 
 13  assembled         1094 non-null   object 
 14  height_mm         1094 non-null   float64
 15  peak_torque_nm    1094 non-null   float64
 16  doors             1094 non-null   float64


## Initialize CarBrand

`CarBrand` class syntax was translated from the Python dataframe using the command below. Then, the output was copied to the application to initialise new instances of `CarBrand`. The output was saved to `init_car_brands.txt`.

Below is the lookup table to transfer from Python syntax to C# syntax.

In [13]:
brand_lookup = {
    "BMW": "BMW", 
    "Ford": "Ford", 
    "Honda": "Honda", 
    "Hyundai": "Hyundai", 
    "Jaguar": "Jaguar", 
    "Kia": "Kia", 
    "Land Rover": "LandRover",
    "Lexus": "Lexus", 
    "MINI": "Mini", 
    "Mazda": "Mazda", 
    "Mercedes-Benz": "MercedesBenz", 
    "Mitsubishi": "Mitsubishi", 
    "Nissan": "Nissan",
    "Perodua": "Perodua", 
    "Peugeot": "Peugeot", 
    "Porsche": "Porsche", 
    "Proton": "Proton", 
    "Renault": "Renault", 
    "Subaru": "Subaru",
    "Toyota": "Toyota", 
    "Volkswagen": "Volkswagen", 
    "Volvo": "Volvo", 
}

In [14]:
car_brands_train = set(car_train['brand'].value_counts().index.tolist())
car_brands_test = set(car_test['brand'].value_counts().index.tolist())
# Set union operation
car_brands = list(car_brands_train.union(car_brands_test))
car_brands.sort(reverse=False)
print(f'Examples of car brand: {car_brands[:10]}')

Examples of car brand: ['BMW', 'Ford', 'Honda', 'Hyundai', 'Jaguar', 'Kia', 'Land Rover', 'Lexus', 'MINI', 'Mazda']


In [15]:
with open(os.path.join(OUT_FOLDER_PATH, 'init_car_brands.txt'), 'w') as f:
    for brand in car_brands:
        brand_init_code = f"""CarBrand {brand_lookup[brand]} = new CarBrand {{ Name=\"{brand}\" }};\n"""
        f.write(f'{brand_init_code}\n')

print(f'Example of C# code initializing CarBrand:\n\n\t{brand_init_code}')

Example of C# code initializing CarBrand:

	CarBrand Volvo = new CarBrand { Name="Volvo" };



## Initialize CarModel

`CarModel` class syntax was translated from the Python dataframe using the command below. Then, the output was copied to the application to initialise new instances of `CarModel`. The output was saved to `init_car_models.txt`.

In [16]:
car_models_train = set(car_train[['brand', 'model']].value_counts().index.tolist())
car_models_test = set(car_test[['brand', 'model']].value_counts().index.tolist())
# Set union operation
car_models = list(car_models_train.union(car_models_test))
car_models.sort(key=lambda tup: tup[0], reverse=False)
print(f'Total number of car model: {len(car_models)}')
print(f'Examples of car model: {car_models[:6]}')

Total number of car model: 360
Examples of car model: [('BMW', '430i'), ('BMW', '540i'), ('BMW', '218i'), ('BMW', '740i'), ('BMW', 'i8'), ('BMW', '318i')]


In [17]:
with open(os.path.join(OUT_FOLDER_PATH, 'init_car_models.txt'), 'w') as f:
    for brand, model in car_models:
        model_name = re.sub(r'[^A-Za-z0-9]', '', model.strip().title())
        brand_name = re.sub(r'[^A-Za-z0-9]', '', brand.strip().title())
        model_init_code = \
        f"""
        CarModel {brand_name}{model_name} = new CarModel
        {{
            Name="{model}",
            CarBrandID={brand_lookup[brand]}.ID,
            CarBrand={brand_lookup[brand]}
        }};
        """
        f.write(f'{model_init_code}\n')

print(f'Example of C# code initializing CarBrand:\n\n\t{model_init_code}')

Example of C# code initializing CarBrand:

	
        CarModel VolvoS60 = new CarModel
        {
            Name="S60",
            CarBrandID=Volvo.ID,
            CarBrand=Volvo
        };
        


## Initialize Car

`Car` class syntax was translated from the Python dataframe using the command below. Then, the output was copied to the application to initialise new instances of `Car`. The output was saved to `init_cars.txt`.

In [18]:
# Load the car price model
with open(os.path.join(OUT_PARENT_DIR, 'car_price', 'trf_rg.pkl'), 'rb') as f:
    car_price_model = pickle.load(f)

with open(os.path.join(OUT_PARENT_DIR, 'car_price', 'data_preprocessor.pkl'), 'rb') as f:
    cp_data_pp = pickle.load(f)

# Get the predicted car price
car_X_test = car_test.drop(columns='price', axis=1)
car_X_test_pp = cp_data_pp.preprocess(car_X_test)
car_test['pred_price'] = car_price_model.predict(car_X_test_pp)

# Randomly assigned values to the 'update_analytics' column
# In the web application, if the users want to update the car price analytics 
# with the current car inventory record, the users must select the option 
# 'Update the current price to price analytics'. 
# The option is randomly chosen during the initialization process to test the system.
ones = np.ones(math.floor(len(car_test)/2))
zeroes = np.zeros(math.ceil(len(car_test)/2))
update_analytics = np.append(ones, zeroes)

# Define a random number generator object
# Source: https://towardsdatascience.com/stop-using-numpy-random-seed-581a9972805f
rng = np.random.default_rng(RANDOM_SEED)
rng.shuffle(update_analytics)

car_test['update_analytics'] = update_analytics

# Export the dataframe to be used in SHAP tree explainer
car_test.to_csv(os.path.join(OUT_FOLDER_PATH, 'car_inventory_records.csv'), index=False)
car_test.head(5)

,manufacture_year,mileage,price,length_mm,engine_cc,aspiration,wheel_base_mm,width_mm,direct_injection,seat_capacity,...,assembled,height_mm,peak_torque_nm,doors,brand,colour,model,transmission,pred_price,update_analytics
0,2006.0,95000.0,12800.0,3480.0,989.0,Aspirated,2360.0,1490.0,Multi-Point Injected,5.0,...,Locally Built,1425.0,88.0,5.0,Perodua,Silver,Kelisa,Automatic,11770.720346,0.0
1,2014.0,75000.0,24800.0,4618.0,1997.0,Aspirated,2702.0,1813.0,Multi-Point Injected,5.0,...,Locally Built,1485.0,195.0,4.0,Renault,Silver,Fluence,Automatic,32159.798111,1.0
2,2014.0,98000.0,86800.0,4705.0,2694.0,Aspirated,2750.0,1840.0,Multi-Point Injected,7.0,...,Locally Built,1850.0,241.0,5.0,Toyota,Silver,Fortuner,Automatic,79405.911083,1.0
3,2003.0,185000.0,8800.0,4465.0,1584.0,Aspirated,2600.0,1740.0,Multi-Point Injected,5.0,...,Locally Built,1420.0,140.0,4.0,Proton,Silver,Waja,Manual,11980.812544,1.0
4,2014.0,105000.0,30800.0,4536.0,1561.0,Turbocharged,2650.0,1786.0,Multi-Point Injected,5.0,...,Locally Built,1524.0,205.0,5.0,Proton,Red,Suprima S,Automatic,31293.387523,0.0


In [19]:
car_test['update_analytics'].value_counts()

0.0    547
1.0    547
Name: update_analytics, dtype: int64

Below is the lookup table to transfer from Python syntax to C# syntax.

In [20]:
cp_table = {
    "ID":{"type":"int"},
    "Aspiration":{
        "Aspirated": "AspirationType.Aspirated", 
        "Supercharged intercooled": "AspirationType.SuperchargedIntercooled", 
        "Turbo intercooled": "AspirationType.TurboIntercooled", 
        "Turbo supercharged intercooled": "AspirationType.TurboSuperchargedIntercooled", 
        "Turbocharged": "AspirationType.Turbocharged", 
        "Twin Turbo intercooled": "AspirationType.TwinTurboIntercooled", 
        "Twin-scroll": "AspirationType.TwinScroll",
        "Twin-scroll turbo": "AspirationType.TwinScrollTurbo",
        "Supercharged": "AspirationType.Supercharged"
    },
    "Assembled":{
        "Locally Built": "AssembledType.LocallyBuilt", 
        "Official Import": "AssembledType.OfficialImport", 
        "Parallel Import": "AssembledType.ParallelImport",
    },
    "Colour": {"type":"string"},
    "DirectInjection":{
        "Direct Injection": "DirectInjectionType.DirectInjection", 
        "Direct/Multi-point injection": "DirectInjectionType.DirectOrMultiPointInjection",
        "Multi-Point Injected": "DirectInjectionType.MultiPointInjected",
        "Carburettor Single": "DirectInjectionType.CarburettorSingle"
    },
    "Doors":{"type":"int"},
    "EngineCC":{"type":"float"},
    "Fuel":{
        "Diesel": "FuelTypeEnum.Diesel",
        "Hybrid": "FuelTypeEnum.Hybrid",
        "Petrol - Unleaded (ULP)": "FuelTypeEnum.UnLeadedPetrol",
        "Petrol - Leaded": "FuelTypeEnum.LeadedPetrol"
    },
    "HeightMM":{"type":"float"},
    "LengthMM":{"type":"float"},
    "ManufactureYear":{"type":"int"},
    "Mileage":{"type":"float"},
    "PeakPowerHP":{"type":"float"},
    "PeakTorqueNM":{"type":"float"},
    "Price":{"type":"float"},
    "PricePerMonth":{"type":"float"},
    "SeatCapacity":{"type":"int"},
    "SteeringType":{
        "Electronic Power Steering": "SteeringTypeEnum.ElectronicPowerSteering", 
        "Hydraulic Power": "SteeringTypeEnum.HydraulicPower", 
        "Rack and Pinion": "SteeringTypeEnum.RackAndPinion",
        "Recirculating Ball": "SteeringTypeEnum.RecirculatingBall", 
        "Worm and Roller": "SteeringTypeEnum.WormAndRoller",
    },
    "Title":{"type":"string"},
    "Transmission":{
        "Automatic": "TransmissionType.Automatic",
        "Manual": "TransmissionType.Manual",
    },
    "UpdateAnalytics" : {
        0: "UpdateAnalyticsEnum.No",
        1: "UpdateAnalyticsEnum.Yes"
    },
    "WheelBaseMM":{"type":"float"},
    "WidthMM":{"type":"float"}
}

In [21]:
car_id = 1
num_of_months = 12

with open(os.path.join(OUT_FOLDER_PATH, 'init_cars.txt'), 'w') as f:
    for index, row in car_test.iterrows():
        model_name = re.sub(r'[^A-Za-z0-9]', '', row['model'].strip().title())
        brand_name = re.sub(r'[^A-Za-z0-9]', '', row['brand'].strip().title())
        car_init_code = f"""
        new Car
        {{
            Aspiration = Car.AspirationTypeList[(int)Car.{cp_table['Aspiration'][row['aspiration']]}],
            Assembled = Car.AssembledTypeList[(int)Car.{cp_table['Assembled'][row['assembled']]}],
            AssignedPrice = {f"{round(row['price'], 2)}m"},
            Colour = "{row['colour'] if row['colour'] != " -" else "null"}",
            CreatedTimestamp = new DateTime(2022, 4, 1, 0, 0, 0),
            DirectInjection = Car.DirectInjectionList[(int)Car.{cp_table['DirectInjection'][row['direct_injection']]}],
            Doors = {int(row['doors'])},
            EngineCC = {row['engine_cc']},
            FuelType = Car.FuelTypeList[(int)Car.{cp_table['Fuel'][row['fuel_type']]}],
            HeightMM = {row['height_mm']},
            LengthMM = {row['length_mm']},
            ManufactureYear = {int(row['manufacture_year'])},
            Mileage = {row['mileage']},
            PeakPowerHP = {row['peak_power_hp']},
            PeakTorqueNM = {row['peak_torque_nm']},
            PredictedPrice = {round(row['pred_price'], 2)}m,
            PricePerMonth = {f"{round(row['price']/num_of_months, 2)}m"},
            SeatCapacity = {int(row['seat_capacity'])},
            SteeringType = Car.SteeringTypeList[(int)Car.{cp_table['SteeringType'][row['steering_type']]}],
            Transmission = Car.TransmissionList[(int)Car.{cp_table['Transmission'][row['transmission']]}],
            UpdateAnalytics = Car.UpdateAnalyticsList[(int)Car.{cp_table['UpdateAnalytics'][row['update_analytics']]}],
            WheelBaseMM = {row['wheel_base_mm']},
            WidthMM = {row['width_mm']},
            CarModelID = {brand_name}{model_name}.ID,
            CarModel = {brand_name}{model_name},
            Title = "{brand_name} {model_name} {car_id}",
        }},
        """
        f.write(f'{car_init_code}\n')
        car_id += 1
        
print(f'Example of C# code initializing Car:\n\n\t{car_init_code}')

Example of C# code initializing Car:

	
        new Car
        {
            Aspiration = Car.AspirationTypeList[(int)Car.AspirationType.Aspirated],
            Assembled = Car.AssembledTypeList[(int)Car.AssembledType.LocallyBuilt],
            AssignedPrice = 7800.0m,
            Colour = "Silver",
            CreatedTimestamp = new DateTime(2022, 4, 1, 0, 0, 0),
            DirectInjection = Car.DirectInjectionList[(int)Car.DirectInjectionType.CarburettorSingle],
            Doors = 5,
            EngineCC = 659.0,
            FuelType = Car.FuelTypeList[(int)Car.FuelTypeEnum.UnLeadedPetrol],
            HeightMM = 1415.0,
            LengthMM = 3365.0,
            ManufactureYear = 2009,
            Mileage = 165000.0,
            PeakPowerHP = 31.0,
            PeakTorqueNM = 49.0,
            PredictedPrice = 6418.17m,
            PricePerMonth = 650.0m,
            SeatCapacity = 5,
            SteeringType = Car.SteeringTypeList[(int)Car.SteeringTypeEnum.RackAndPinion],
       